In [1]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error,r2_score
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0700.hk')
data.drop("Adj Close",axis=1,inplace=True)
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2004-06-16,0.875,0.925,0.815,0.830,2198875000
2004-06-17,0.830,0.875,0.825,0.845,419007500
2004-06-18,0.840,0.850,0.790,0.805,182990000
2004-06-21,0.820,0.825,0.790,0.800,114085000
2004-06-22,0.800,0.800,0.800,0.800,0


In [3]:

#Create one more column Prediction shifted 15 days up. 
data['Prediction'] = data[['Close']].shift(-1)
data.dropna(inplace=True)
df=data[:-200]
#print data set
print(df)

               Open     High      Low    Close      Volume  Prediction
Date                                                                  
2004-06-16    0.875    0.925    0.815    0.830  2198875000       0.845
2004-06-17    0.830    0.875    0.825    0.845   419007500       0.805
2004-06-18    0.840    0.850    0.790    0.805   182990000       0.800
2004-06-21    0.820    0.825    0.790    0.800   114085000       0.800
2004-06-22    0.800    0.800    0.800    0.800           0       0.885
...             ...      ...      ...      ...         ...         ...
2021-03-01  692.000  697.000  679.000  697.000    27079038     698.000
2021-03-02  712.000  722.000  692.500  698.000    23080003     723.000
2021-03-03  698.000  723.000  690.000  723.000    18285294     690.000
2021-03-04  702.000  709.500  690.000  690.000    24497606     679.000
2021-03-05  668.500  695.000  667.000  679.000    27137066     642.500

[4133 rows x 6 columns]


In [4]:
#Create a data set X and convert it into numpy array , which will be having actual values
X = df.drop(['Prediction'],1).values.astype(float)
print(X)
# Create a dataset y which will be having Predicted values and convert into numpy array
y = df['Prediction'].values.astype(float)
print(y)

[[8.75000000e-01 9.25000012e-01 8.14999998e-01 8.29999983e-01
  2.19887500e+09]
 [8.29999983e-01 8.75000000e-01 8.24999988e-01 8.45000029e-01
  4.19007500e+08]
 [8.39999974e-01 8.50000024e-01 7.90000021e-01 8.05000007e-01
  1.82990000e+08]
 ...
 [6.98000000e+02 7.23000000e+02 6.90000000e+02 7.23000000e+02
  1.82852940e+07]
 [7.02000000e+02 7.09500000e+02 6.90000000e+02 6.90000000e+02
  2.44976060e+07]
 [6.68500000e+02 6.95000000e+02 6.67000000e+02 6.79000000e+02
  2.71370660e+07]]
[  0.84500003   0.80500001   0.80000001 ... 690.         679.
 642.5       ]


In [5]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
# X = sc_X.fit_transform(X)
# y = sc_y.fit_transform(y.reshape(-1,1))

In [6]:
# Split the data into train and test with 90 & 10 % respectively
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [7]:
# SVM Model
svr = SVR()
# Train the model 
svr.fit(x_train, y_train)

SVR()

In [8]:
# The best possible score is 1.0
svm_confidence = svr.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  -0.29238417630104463


In [9]:
y_pred=svr.predict(x_test).reshape(-1,1)
y_pred = sc_y.inverse_transform(y_pred) 
y_test=sc_y.inverse_transform(y_test)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

# Unseen Data

In [ ]:
forecast_data = np.array(data.drop(['Prediction'],1))[-200:]
forecast = np.array(data['Close'])[-200:].reshape(-1,1)
print(forecast)

In [ ]:
# support vector model predictions for the next ‘15’ days
svm_prediction = svr.predict(forecast_data).reshape(-1,1)
svm_prediction = sc_y.inverse_transform(svm_prediction) 
print(svm_prediction)

In [ ]:
mean_absolute_percentage_error(forecast,svm_prediction)*100